In [1]:
#import os
import pandas as pd

#Moving average smoothing
def moving_average(data, window_size):
    smoothed_data = []
    for i in range(len(data) - window_size + 1):
        window_data = data[i : i + window_size]
        window_avg = sum(window_data) / window_size
        smoothed_data.append(window_avg)
    return smoothed_data







In [2]:
#Calculation of Classical Statistics
def statistic(data):
    #smoothing
    data1 = moving_average(data, 3)
    len_ = len(data1)
    mean_ = np.mean(data1) 
    std_ = np.std(data1) 
    var_ = np.var(data1)
    max_ = max(data1) 
    min_ = min(data1) 
    pk_ = max(abs(max_),abs(min_))
    mid_ = np.median(data1) 
    rms_=0
    abs_xbar=0
    r_ = 0
    s = 0
    k = 0
    for i in range(len_):
        rms_ += data1[i] ** 2
        abs_xbar += abs(data1[i])
        r_ += math.sqrt(abs(data1[i]))
        s += (data1[i] - mean_) ** 3
        k += (data1[i] - mean_) ** 4
    rms_ = math.sqrt(rms_ / len_)  
    abs_xbar = abs_xbar / len_
    r_ = (r_ / len_) ** 2  # Root Mean Square Amplitude
    s_ = s / ((len_ - 1) * std_ ** 3)  # Skewness
    k_ = k / ((len_ - 1) * std_ ** 4)  # Kurtosis
    return mean_, var_, std_,  max_, min_, pk_, mid_, rms_, abs_xbar, r_, s_, k_

# Extraction of Kinematic Parameters from MOCAP

In [3]:
import numpy as np
import math

#Instantaneous speed
def mocap_speed(data): 
    
  
    delta_x = np.diff(data['X19'])
    delta_y = np.diff(data['Y19'])
    delta_z = np.diff(data['Z19'])

    delta_t = np.diff(data['Time'])
    velocity_x = abs(delta_x / delta_t)
    velocity_y = abs(delta_y / delta_t)
    velocity_z = abs(delta_z / delta_t)
    return velocity_x, velocity_y, velocity_z

#Knee Joint Angle, Ankle Joint Angle
def mocap_angle(data):
    #Points 13, 15, and 17 define the knee joint angle
    knee13_15=data[['X13','Y13','Z13']].values-data[['X15','Y15','Z15']].values
    knee15_17=data[['X17','Y17','Z17']].values-data[['X15','Y15','Z15']].values
   
    mod_knee13_15 = np.linalg.norm(knee13_15, axis=1)
    mod_knee15_17 = np.linalg.norm(knee15_17, axis=1)
  
    dot_knee = (knee13_15 * knee15_17).sum(axis=1)
    
    cos_knee = dot_knee / (mod_knee13_15 * mod_knee15_17)
    
    #Calculate the radian and degree values of the angle
    theta_knee = np.arccos(cos_knee)
    theta_deg_knee = np.rad2deg(theta_knee)
    
    #Points 17, 19, and 21 define the ankle joint angle
    ankle17_19=data[['X17','Y17','Z17']].values-data[['X19','Y19','Z19']].values
    ankle19_21=data[['X21','Y21','Z21']].values-data[['X19','Y19','Z19']].values
    
  
    mod_ankle17_19 = np.linalg.norm(ankle17_19, axis=1)
    mod_ankle19_21 = np.linalg.norm(ankle19_21, axis=1)
   
    dot_ankle = (ankle17_19 * ankle19_21).sum(axis=1)
   
    cos_ankle = dot_ankle / (mod_ankle17_19 * mod_ankle19_21)
    
    #Calculate the radian and degree values of the angle
    theta_ankle = np.arccos(cos_ankle)
    theta_deg_ankle = np.rad2deg(theta_ankle)
    
    return theta_deg_knee,theta_deg_ankle



In [13]:
import os
#Extract kinematic parameters from mocap”
def mocap_ki_parameter(file_path):
    mocap_kinematic = pd.DataFrame(columns=['device','sample', 'times', 'process', 'task', 
                                       'Xvel_mean','Xvel_var', 'Xvel_std', 'Xvel_max', 'Xvel_min', 'Xvel_pk', 'Xvel_mid','Xvel_rms','Xvel_abs_xbar','Xvel_r','Xvel_S','Xvel_K',
                                       'Yvel_mean','Yvel_var', 'Yvel_std', 'Yvel_max', 'Yvel_min', 'Yvel_pk', 'Yvel_mid','Yvel_rms','Yvel_abs_xbar','Yvel_r','Yvel_S','Yvel_K',
                                       'Zvel_mean','Zvel_var', 'Zvel_std', 'Zvel_max', 'Zvel_min', 'Zvel_pk', 'Zvel_mid','Zvel_rms','Zvel_abs_xbar','Zvel_r','Zvel_S','Zvel_K',
                                       'knee_mean','knee_var', 'knee_std', 'knee_max', 'knee_min', 'knee_pk', 'knee_mid','knee_rms','knee_abs_xbar','knee_r','knee_S','knee_K',
                                       'ankle_mean','ankle_var', 'ankle_std', 'ankle_max', 'ankle_min', 'ankle_pk', 'ankle_mid','ankle_rms','ankle_abs_xbar','ankle_r','ankle_S','ankle_K'
                                       ])
    t = 0
    #Getting all files and folders in a specified directory
    contents = os.listdir(file_path)
    for item in contents[:]:
        item_path = os.path.join(file_path, item)  
        print('folder:' + item_path)
        content_item = os.listdir(item_path)

        # Traverse the folders 'go'/'back'/'turn'
        for process in content_item[:]:
            process_path = os.path.join(item_path, process)
            print('folder:' + process_path)
            process_contents = os.listdir(process_path)

            # Traverse the folders 'task'
            for task in process_contents:
                task_path = os.path.join(process_path, task)
                print("folder:" + task_path)
                task_contents = os.listdir(task_path)
                # Traverse the files 'sample'
                for num in task_contents:
                    num_path = os.path.join(task_path, num)
                    print("file:" + num_path)
                    
                    num1 = pd.read_csv(num_path)
                    if len(num1)<1:
                        continue
                
                    speed_x,speed_y,speed_z = mocap_speed(num1)
                    knee,ankle = mocap_angle(num1)
                    
                    
                    xspeed_mean,xspeed_var, xspeed_std, xspeed_max, xspeed_min, xspeed_pk, xspeed_mid,xspeed_rms,xspeed_abs_xbar,xspeed_r,xspeed_s,xspeed_k=statistic(speed_x)
                    yspeed_mean,yspeed_var, yspeed_std, yspeed_max, yspeed_min, yspeed_pk, yspeed_mid,yspeed_rms,yspeed_abs_xbar,yspeed_r,yspeed_s,yspeed_k=statistic(speed_y)
                    zspeed_mean,zspeed_var, zspeed_std, zspeed_max, zspeed_min, zspeed_pk, zspeed_mid,zspeed_rms,zspeed_abs_xbar,zspeed_r,zspeed_s,zspeed_k=statistic(speed_z)

                    knee_mean,knee_var, knee_std, knee_max, knee_min, knee_pk, knee_mid,knee_rms,knee_abs_xbar,knee_r,knee_s,knee_k=statistic(knee)
                     
                    ankle_mean,ankle_var, ankle_std, ankle_max, ankle_min, ankle_pk, ankle_mid,ankle_rms,ankle_abs_xbar,ankle_r,ankle_s,ankle_k=statistic(ankle)
                    
                    
                    
                    mocap_kinematic.loc[t] = ['MOCAP',num[:8], item, process, task,xspeed_mean,xspeed_var, xspeed_std, xspeed_max, xspeed_min, xspeed_pk, xspeed_mid,xspeed_rms,xspeed_abs_xbar,xspeed_r,xspeed_s,xspeed_k,yspeed_mean,yspeed_var, yspeed_std, yspeed_max, yspeed_min, yspeed_pk, yspeed_mid,yspeed_rms,yspeed_abs_xbar,yspeed_r,yspeed_s,yspeed_k,zspeed_mean,zspeed_var, zspeed_std, zspeed_max, zspeed_min, zspeed_pk, zspeed_mid,zspeed_rms,zspeed_abs_xbar,zspeed_r,zspeed_s,zspeed_k, knee_mean,knee_var, knee_std, knee_max, knee_min, knee_pk, knee_mid,knee_rms,knee_abs_xbar,knee_r,knee_s,knee_k,ankle_mean,ankle_var, ankle_std, ankle_max, ankle_min, ankle_pk, ankle_mid,ankle_rms,ankle_abs_xbar,ankle_r,ankle_s,ankle_k]
                    
                    t = t + 1

    mocap_kinematic.to_csv('\Data_new\\Parameters\\MOCAP_5m\\kinematic_parameters.csv',index = False)





In [14]:
mocap_ki_parameter("\Data_new\\Processed\\MOCAP_5m")

folder:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\MOCAP_5m\time01
folder:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\MOCAP_5m\time01\back
folder:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\MOCAP_5m\time01\back\task_cup
file:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\MOCAP_5m\time01\back\task_cup\sample01_task_cup.csv
file:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\MOCAP_5m\time01\back\task_cup\sample02_task_cup.csv
file:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\MOCAP_5m\time01\back\task_cup\sample03_task_cup.csv
file:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\MOCAP_5m\time01\back\task_cup\sample04_task_cup.csv
file:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\MOCAP_5m\time01\back\task_cup\sample05_task_cup.csv
file:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\MOCAP_5m\time01\back\task_cup\sample06_task_cup.csv
file:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Proce

# Extraction of Kinematic Parameters from Kinect

In [6]:

import numpy as np
import math


#Speed
def kinect_speed(data):
  
    delta_x = np.diff(data['X14'])
    delta_y = np.diff(data['Y14'])
    delta_z = np.diff(data['Z14'])

    delta_t = np.diff(data['Timestamp'])

    velocity_x = abs(delta_x / delta_t)
    velocity_y = abs(delta_y / delta_t)
    velocity_z = abs(delta_z / delta_t)
    return velocity_x, velocity_y, velocity_z

#Knee Joint Angle, Ankle Joint Angle
def kinect_angle(data):
    #Points 12, 13, and 14 define the knee joint angle
    knee12_13=data[['X12','Y12','Z12']].values-data[['X13','Y13','Z13']].values
    knee13_14=data[['X14','Y14','Z14']].values-data[['X13','Y13','Z13']].values
    
    mod_knee12_13 = np.linalg.norm(knee12_13, axis=1)
    mod_knee13_14 = np.linalg.norm(knee13_14, axis=1)
    
    dot_knee = (knee12_13 * knee13_14).sum(axis=1)
   
    cos_knee = dot_knee / (mod_knee12_13 * mod_knee13_14)
    
  
    theta_knee = np.arccos(cos_knee)
    theta_deg_knee = np.rad2deg(theta_knee)
    

    #Points 13, 14, and 15 define the ankle joint angle
    ankle13_14=data[['X13','Y13','Z13']].values-data[['X14','Y14','Z14']].values
    ankle14_15=data[['X15','Y15','Z15']].values-data[['X14','Y14','Z14']].values
    
    mod_ankle13_14 = np.linalg.norm(ankle13_14, axis=1)
    mod_ankle14_15 = np.linalg.norm(ankle14_15, axis=1)

    dot_ankle = (ankle13_14 * ankle14_15).sum(axis=1)

    cos_ankle = dot_ankle / (mod_ankle13_14 * mod_ankle14_15)
    
    theta_ankle = np.arccos(cos_ankle)
    theta_deg_ankle = np.rad2deg(theta_ankle)
    
    return theta_deg_knee,theta_deg_ankle


In [7]:
#Extract kinematic parameters from Kinect
def kinect_ki_parameter(file_path):
    kinect_kinematic = pd.DataFrame(columns=['device','sample', 'times', 'process', 'task', 
                                       'Xvel_mean','Xvel_var', 'Xvel_std', 'Xvel_max', 'Xvel_min', 'Xvel_pk', 'Xvel_mid','Xvel_rms','Xvel_abs_xbar','Xvel_r','Xvel_S','Xvel_K',
                                       'Yvel_mean','Yvel_var', 'Yvel_std', 'Yvel_max', 'Yvel_min', 'Yvel_pk', 'Yvel_mid','Yvel_rms','Yvel_abs_xbar','Yvel_r','Yvel_S','Yvel_K',
                                       'Zvel_mean','Zvel_var', 'Zvel_std', 'Zvel_max', 'Zvel_min', 'Zvel_pk', 'Zvel_mid','Zvel_rms','Zvel_abs_xbar','Zvel_r','Zvel_S','Zvel_K',
                                       'knee_mean','knee_var', 'knee_std', 'knee_max', 'knee_min', 'knee_pk', 'knee_mid','knee_rms','knee_abs_xbar','knee_r','knee_S','knee_K',
                                       'ankle_mean','ankle_var', 'ankle_std', 'ankle_max', 'ankle_min', 'ankle_pk', 'ankle_mid','ankle_rms','ankle_abs_xbar','ankle_r','ankle_S','ankle_K'
                                       ])
    t = 0
    #Getting all files and folders in a specified directory
    contents = os.listdir(file_path)
    for item in contents[:]:
        item_path = os.path.join(file_path, item)  
        print('folder:' + item_path)
        content_item = os.listdir(item_path)

        # go/back/turn
        for process in content_item[:]:
            process_path = os.path.join(item_path, process)
            print('folder:' + process_path)
            process_contents = os.listdir(process_path)

            # task
            for task in process_contents:
                task_path = os.path.join(process_path, task)
                print("folder:" + task_path)
                task_contents = os.listdir(task_path)
                # file 'sample'
                for num in task_contents:
                    num_path = os.path.join(task_path, num)
                    print("file" + num_path)
                    
                    num1 = pd.read_csv(num_path)
                    if len(num1)<1:
                        continue

   
                    speed_x,speed_y,speed_z = kinect_speed(num1)  
                    knee,ankle=kinect_angle(num1)
                    
                    
                    xspeed_mean,xspeed_var, xspeed_std, xspeed_max, xspeed_min, xspeed_pk, xspeed_mid,xspeed_rms,xspeed_abs_xbar,xspeed_r,xspeed_s,xspeed_k=statistic(speed_x)
                    yspeed_mean,yspeed_var, yspeed_std, yspeed_max, yspeed_min, yspeed_pk, yspeed_mid,yspeed_rms,yspeed_abs_xbar,yspeed_r,yspeed_s,yspeed_k=statistic(speed_y)
                    zspeed_mean,zspeed_var, zspeed_std, zspeed_max, zspeed_min, zspeed_pk, zspeed_mid,zspeed_rms,zspeed_abs_xbar,zspeed_r,zspeed_s,zspeed_k=statistic(speed_z)
                    
                    knee_mean,knee_var, knee_std, knee_max, knee_min, knee_pk, knee_mid,knee_rms,knee_abs_xbar,knee_r,knee_s,knee_k=statistic(knee)
                     
                    ankle_mean,ankle_var, ankle_std, ankle_max, ankle_min, ankle_pk, ankle_mid,ankle_rms,ankle_abs_xbar,ankle_r,ankle_s,ankle_k=statistic(ankle)
                    
                    kinect_kinematic.loc[t] = ['Kinect',num[:8], item, process, task,xspeed_mean,xspeed_var, xspeed_std, xspeed_max, xspeed_min, xspeed_pk, xspeed_mid,xspeed_rms,xspeed_abs_xbar,xspeed_r,xspeed_s,xspeed_k,yspeed_mean,yspeed_var, yspeed_std, yspeed_max, yspeed_min, yspeed_pk, yspeed_mid,yspeed_rms,yspeed_abs_xbar,yspeed_r,yspeed_s,yspeed_k,zspeed_mean,zspeed_var, zspeed_std, zspeed_max, zspeed_min, zspeed_pk, zspeed_mid,zspeed_rms,zspeed_abs_xbar,zspeed_r,zspeed_s,zspeed_k,knee_mean,knee_var, knee_std, knee_max, knee_min, knee_pk, knee_mid,knee_rms,knee_abs_xbar,knee_r,knee_s,knee_k,ankle_mean,ankle_var, ankle_std, ankle_max, ankle_min, ankle_pk, ankle_mid,ankle_rms,ankle_abs_xbar,ankle_r,ankle_s,ankle_k]
                    t = t + 1

    kinect_kinematic.to_csv('\Data_new\\Parameters\\Kinect_2m\\kinematic_parameters.csv',index = False)





In [8]:
kinect_ki_parameter("\Data_new\\Processed\\Kinect_2m")

folder:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\Kinect_2m\time01
folder:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\Kinect_2m\time01\back
folder:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\Kinect_2m\time01\back\task_cup
fileD:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\Kinect_2m\time01\back\task_cup\sample01_task_cup.csv
fileD:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\Kinect_2m\time01\back\task_cup\sample02_task_cup.csv
fileD:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\Kinect_2m\time01\back\task_cup\sample03_task_cup.csv
fileD:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\Kinect_2m\time01\back\task_cup\sample04_task_cup.csv
fileD:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\Kinect_2m\time01\back\task_cup\sample05_task_cup.csv
fileD:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\Kinect_2m\time01\back\task_cup\sample06_task_cup.csv
fileD:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Pro

# Extraction of Kinematic Parameters from IMU

In [11]:

def imu_ki_parameter(file_path):
    imu_kinematic = pd.DataFrame(columns=['device','sample', 'times', 'process', 'task', 
                                       'Xacc_mean','Xacc_var', 'Xacc_std', 'Xacc_max', 'Xacc_min', 'Xacc_pk', 'Xacc_mid','Xacc_rms','Xacc_abs_xbar','Xacc_r','Xacc_S','Xacc_K',
                                       'Yacc_mean','Yacc_var', 'Yacc_std', 'Yacc_max', 'Yacc_min', 'Yacc_pk', 'Yacc_mid','Yacc_rms','Yacc_abs_xbar','Yacc_r','Yacc_S','Yacc_K',
                                       'Zacc_mean','Zacc_var', 'Zacc_std', 'Zacc_max', 'Zacc_min', 'Zacc_pk', 'Zacc_mid','Zacc_rms','Zacc_abs_xbar','Zacc_r','Zacc_S','Zacc_K',
                                       'Xav_mean','Xav_var', 'Xav_std', 'Xav_max', 'Xav_min', 'Xav_pk', 'Xav_mid','Xav_rms','Xav_abs_xbar','Xav_r','Xav_S','Xanv_K',
                                       'Yav_mean','Yav_var', 'Yav_std', 'Yav_max','Yav_min', 'Yav_pk', 'Yav_mid', 'Yav_rms','Yav_abs_xbar','Yav_r','Yav_S','Yanv_K',
                                       'Zav_mean','Zav_var', 'Zav_std', 'Zav_max', 'Zav_min', 'Zav_pk', 'Zav_mid','Zav_rms','Zav_abs_xbar','Zav_r','Zav_S','Zanv_K',
                                       'Xangle_mean','Xangle_var', 'Xangle_std', 'Xangle_max', 'Xangle_min', 'Xangle_pk', 'Xangle_mid','Xangle_rms','Xangle_abs_xbar','Xangle_r','Xangle_S','Xangle_K',
                                       'Yangle_mean','Yangle_var', 'Yangle_std', 'Yangle_max', 'Yangle_min', 'Yangle_pk', 'Yangle_mid','Yangle_rms','Yangle_abs_xbar','Yangle_r','Yangle_S','Yangle_K',
                                       'Zangle_mean','Zangle_var', 'Zangle_std', 'Zangle_max', 'Zangle_min', 'Zangle_pk', 'Zangle_mid','Zangle_rms','Zangle_abs_xbar','Zangle_r','Zangle_S','Zangle_K'
                                       ])
    t = 0

    contents = os.listdir(file_path)
    for item in contents[:]:
        item_path = os.path.join(file_path, item)  
        print('folder:' + item_path)
        content_item = os.listdir(item_path)


        for process in content_item[:]:
            process_path = os.path.join(item_path, process)
            print('folder:' + process_path)
            process_contents = os.listdir(process_path)

        
            for task in process_contents:
                task_path = os.path.join(process_path, task)
                print("folder:" + task_path)
                task_contents = os.listdir(task_path)
              
                for num in task_contents:
                    num_path = os.path.join(task_path, num)
                    print("file:" + num_path)
                    
                    num1 = pd.read_csv(num_path)
                    if len(num1)<1:
                        continue
                    
                    xacc_mean,xacc_var, xacc_std, xacc_max, xacc_min, xacc_pk, xacc_mid,xacc_rms,xacc_abs_xbar,xacc_r,xacc_s,xacc_k= statistic(abs(num1['ACC_X']*9.8))                 
                    yacc_mean,yacc_var, yacc_std, yacc_max, yacc_min, yacc_pk, yacc_mid,yacc_rms,yacc_abs_xbar,yacc_r,yacc_s,yacc_k=statistic(abs(num1['ACC_Y']*9.8))   
                    zacc_mean,zacc_var, zacc_std, zacc_max, zacc_min, zacc_pk, zacc_mid,zacc_rms,zacc_abs_xbar,zacc_r,zacc_s,zacc_k=statistic(abs(num1['ACC_Z']*9.8))   
                    
                    
                    xangle_mean,xangle_var, xangle_std, xangle_max, xangle_min, xangle_pk, xangle_mid,xangle_rms,xangle_abs_xbar,xangle_r,xangle_s,xangle_k=statistic(abs(num1['Angle_X']))
                    yangle_mean,yangle_var, yangle_std, yangle_max, yangle_min, yangle_pk, yangle_mid,yangle_rms,yangle_abs_xbar,yangle_r,yangle_s,yangle_k=statistic(abs(num1['Angle_Y']))
                    zangle_mean,zangle_var, zangle_std, zangle_max, zangle_min, zangle_pk, zangle_mid,zangle_rms,zangle_abs_xbar,zangle_r,zangle_s,zangle_k=statistic(abs(num1['Angle_Z']))
                    
                    
                    xanv_mean,xanv_var, xanv_std, xanv_max, xanv_min, xanv_pk, xanv_mid,xanv_rms,xanv_abs_xbar,xanv_r,xanv_s,xanv_k=statistic(abs(num1['AV_X']))
                    yanv_mean,yanv_var, yanv_std, yanv_max,yanv_min, yanv_pk, yanv_mid, yanv_rms,yanv_abs_xbar,yanv_r,yanv_s,yanv_k=statistic(abs(num1['AV_Y']))
                    zanv_mean,zanv_var, zanv_std, zanv_max, zanv_min,zanv_pk, zanv_mid,zanv_rms,zanv_abs_xbar,zanv_r,zanv_s,zanv_k=statistic(abs(num1['AV_Z']))

                    
                    imu_kinematic.loc[t] = ['IMU',num[:8], item, process, task,xacc_mean,xacc_var, xacc_std, xacc_max, xacc_min, xacc_pk, xacc_mid,xacc_rms,xacc_abs_xbar,xacc_r,xacc_s,xacc_k,yacc_mean,yacc_var, yacc_std, yacc_max, yacc_min, yacc_pk, yacc_mid,yacc_rms,yacc_abs_xbar,yacc_r,yacc_s,yacc_k, zacc_mean,zacc_var, zacc_std, zacc_max, zacc_min, zacc_pk, zacc_mid,zacc_rms,zacc_abs_xbar,zacc_r,zacc_s,zacc_k,xanv_mean,xanv_var, xanv_std, xanv_max, xanv_min, xanv_pk, xanv_mid,xanv_rms,xanv_abs_xbar,xanv_r,xanv_s,xanv_k,yanv_mean,yanv_var, yanv_std, yanv_max,yanv_min, yanv_pk, yanv_mid, yanv_rms,yanv_abs_xbar,yanv_r,yanv_s,yanv_k,zanv_mean,zanv_var, zanv_std, zanv_max, zanv_min,zanv_pk, zanv_mid,zanv_rms,zanv_abs_xbar,zanv_r,zanv_s,zanv_k,xangle_mean,xangle_var, xangle_std, xangle_max, xangle_min, xangle_pk, xangle_mid,xangle_rms,xangle_abs_xbar,xangle_r,xangle_s,xangle_k,yangle_mean,yangle_var, yangle_std, yangle_max, yangle_min, yangle_pk, yangle_mid,yangle_rms,yangle_abs_xbar,yangle_r,yangle_s,yangle_k,zangle_mean,zangle_var, zangle_std, zangle_max, zangle_min, zangle_pk, zangle_mid,zangle_rms,zangle_abs_xbar,zangle_r,zangle_s,zangle_k]
                    t = t + 1

    imu_kinematic.to_csv('\Data_new\\Parameters\\IMU_2m\\kinematic_parameters.csv',index = False)
    #imu_kinematic.to_csv('\\Data\\Parameters\\IMU_2m\\kinematic_parameters.csv',index = False)






In [12]:
imu_ki_parameter("\Data_new\\Processed\\IMU_2m")
#imu_ki_parameter("\\Data\\Processed\\IMU_2m")

folder:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\IMU_2m\time01
folder:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\IMU_2m\time01\back
folder:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\IMU_2m\time01\back\task_cup
file:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\IMU_2m\time01\back\task_cup\sample01_task_cup.csv
file:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\IMU_2m\time01\back\task_cup\sample02_task_cup.csv
file:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\IMU_2m\time01\back\task_cup\sample03_task_cup.csv
file:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\IMU_2m\time01\back\task_cup\sample04_task_cup.csv
file:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\IMU_2m\time01\back\task_cup\sample05_task_cup.csv
file:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\IMU_2m\time01\back\task_cup\sample06_task_cup.csv
file:D:\A-lyy\论文\多元传感在步态分析中的应用\论文\提交数据集和代码\Data_new\Processed\IMU_2m\time01